In [1]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-
'''

This process trains regression models of consumer expenditure based on 
demographic properties from the BLS Consumer Expenditure Survey.

    Independent variables are calculated including bins, indicators to help tree based methods
    A model is created for each 
        race 'white','black','asian','other','hispanic'
        and subset of spending 'TOTEXP','FOOD','HOUS','TRANS','HEALTH','RETPEN'
    During prediction an additional property is calculated as the difference between the total and subsets 'OTHER'
    Finally, the list of models is saved to a pickle file for the next process, penetration.
    
'''
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
import warnings
warnings.filterwarnings("ignore")

import xgboost as xgb
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

dependents = ['TOTEXP','FOOD','ALCBEV','HOUS','APPAR','TRANS','HEALTH', 'ENTERT',
              'PERSCA', 'READ','EDUCA','TOBACC','MISC', 'CASHCO','RETPEN']

independents = ['income_z',
                'income_A', 'income_B', 'income_C', 'income_D', 'income_E', 'income_F', 'income_G', 
                'homeownership',
                'vehicles_0_1',
                'vehicles_0', 'vehicles_1', 'vehicles_2', 'vehicles_3', 'vehicles_4', 'vehicles_5plus',
                'married',
                'children',
                'bedroomstwofewer',
                'bedrooms_0', 'bedrooms_1', 'bedrooms_2', 'bedrooms_3', 'bedrooms_4', 'bedrooms_5plus',
                'fuel_log',
                'fuel_A', 'fuel_B', 'fuel_C', 'fuel_D', 'fuel_E', 'fuel_F', 'fuel_G',
                'familysize_1','familysize_2','familysize_3','familysize_4','familysize_5','familysize_6','familysize_7more',
                #'income_low','income_lowermid','income_uppermid','income_high',
               ]


#
# Prepare the FMLI for training, adding derived variable, addressing NAs etc.
#

fmli = pd.read_pickle('fmli.pkl')

# Incomplete interviews
fmli = fmli[fmli.TOTEXP > 0].copy()            # 1627 rows removed, incomplete surveys
fmli = fmli[fmli.FINCBTXM > 0].copy()          # 70 rows removed
fmli = fmli[~fmli.BEDROOMQ.isna()].copy()      # 68 rows have NA
fmli[fmli.HEALTH>=0].copy()                    # 44 rows are negative

fmli = fmli[(fmli['NTLGASCQ'] + fmli['NTLGASPQ'] + # 1654 rows have no fuel spending
             fmli['ELCTRCCQ'] + fmli['ELCTRCPQ'] + fmli['ALLFULCQ'] + fmli['ALLFULPQ'])>0]

# Enforce integers
for var in ['REF_RACE','HISP_REF','TOTEXP','FINCBTXM','CUTENURE','BEDROOMQ','VEHQ','FAM_TYPE',
            'NTLGASCQ','NTLGASPQ','ELCTRCCQ','ELCTRCPQ','ALLFULCQ','ALLFULPQ',]:
    fmli[var] = fmli[var].astype(int)

# Creating a model for each race, with hispanic like another race
fmli['model_white'] = np.where((fmli.REF_RACE == 1) & (fmli.HISP_REF == 2),1,0)
fmli['model_black'] = np.where((fmli.REF_RACE == 2) & (fmli.HISP_REF == 2),1,0)
fmli['model_asian'] = np.where((fmli.REF_RACE == 4) & (fmli.HISP_REF == 2),1,0)
fmli['model_other'] = np.where((~fmli.REF_RACE.isin([1,2,4])) & (fmli.HISP_REF == 2),1,0)
fmli['model_hispanic'] = np.where(fmli.HISP_REF == 1,1,0)

# Calculate the dependent variables
fmli['income_log'] = np.log(fmli.FINCBTXM)
fmli['income_z'] = (fmli.income_log-fmli.income_log.mean())/fmli.income_log.std()
fmli['homeownership'] = (fmli.CUTENURE.astype(int) < 4).astype(int) 
fmli['vehicles_0_1'] = (fmli.VEHQ < 2).astype(int)
fmli['vehicles_0'] = np.where(fmli.VEHQ == 0,1,0)
fmli['vehicles_1'] = np.where(fmli.VEHQ == 1,1,0)
fmli['vehicles_2'] = np.where(fmli.VEHQ == 2,1,0)
fmli['vehicles_3'] = np.where(fmli.VEHQ == 3,1,0)
fmli['vehicles_4'] = np.where(fmli.VEHQ == 4,1,0)
fmli['vehicles_5plus'] = np.where(fmli.VEHQ >= 5,1,0)
fmli["married"] = (fmli.FAM_TYPE < 6).astype(int)
fmli["children"] = (fmli.FAM_TYPE.isin([2,3,4,6,7])).astype(int)
fmli['bedroomstwofewer'] = np.where(fmli.BEDROOMQ<3,1,0)
fmli['bedrooms_0'] = np.where(fmli.BEDROOMQ == 0,1,0)
fmli['bedrooms_1'] = np.where(fmli.BEDROOMQ == 1,1,0)
fmli['bedrooms_2'] = np.where(fmli.BEDROOMQ == 2,1,0)
fmli['bedrooms_3'] = np.where(fmli.BEDROOMQ == 3,1,0)
fmli['bedrooms_4'] = np.where(fmli.BEDROOMQ == 4,1,0)
fmli['bedrooms_5plus'] = np.where(fmli.BEDROOMQ >= 5,1,0)
fmli['fuel_log'] = np.log(1+fmli['NTLGASCQ']+fmli['NTLGASPQ']+fmli['ELCTRCCQ'] + fmli['ELCTRCPQ']+fmli['ALLFULCQ']+fmli['ALLFULPQ'])
fmli['fuel_z'] = (fmli['fuel_log']-fmli['fuel_log'].mean()) / fmli['fuel_log'].std()
fmli['familysize_1'] = np.where(fmli.FAM_SIZE == 1, 1, 0)
fmli['familysize_2'] = np.where(fmli.FAM_SIZE == 2, 1, 0)
fmli['familysize_3'] = np.where(fmli.FAM_SIZE == 3, 1, 0)
fmli['familysize_4'] = np.where(fmli.FAM_SIZE == 4, 1, 0)
fmli['familysize_5'] = np.where(fmli.FAM_SIZE == 5, 1, 0)
fmli['familysize_6'] = np.where(fmli.FAM_SIZE == 6, 1, 0)
fmli['familysize_7more'] = np.where(fmli.FAM_SIZE >= 7, 1, 0)
# Using income brackets did not improve predictiveness
#fmli["income_low"]      = np.where(fmli.FINCBTXM <  35000,1,0)
#fmli["income_lowermid"] = np.where((fmli.FINCBTXM >= 35000)&(fmli.FINCBTXM <  75000),1,0)
#fmli["income_uppermid"] = np.where((fmli.FINCBTXM >= 75000)&(fmli.FINCBTXM < 130000),1,0)
#fmli["income_high"]     = np.where(fmli.FINCBTXM >= 130000,1,0)
fmli = pd.concat( \
    [fmli.reset_index(drop=True),
     pd.get_dummies(\
                     np.where(fmli['income_z'] < -1.5, 'A',
                     np.where(fmli['income_z'] < -.75, 'B',
                     np.where(fmli['income_z'] < -.25, 'C',
                     np.where(fmli['income_z'] < .25, 'D',
                     np.where(fmli['income_z'] < .75, 'E', 
                     np.where(fmli['income_z'] < 1.5, 'F', 'G')))))),prefix="income", dtype=int).reset_index(drop=True),
    pd.get_dummies(\
                     np.where(fmli['fuel_z'] < -1.5, 'A',
                     np.where(fmli['fuel_z'] < -.75, 'B',
                     np.where(fmli['fuel_z'] < -.25, 'C',
                     np.where(fmli['fuel_z'] < .25, 'D',
                     np.where(fmli['fuel_z'] < .75, 'E', 
                     np.where(fmli['fuel_z'] < 1.5, 'F', 'G')))))),prefix="fuel", dtype=int).reset_index(drop=True),
    ], axis=1)

print("Ready")

Ready


In [2]:
#
# Regression models
#


# The models will be kept in a dictionary of dictionaries
races = ['white','black','asian','other','hispanic',]
targets = ['TOTEXP','FOOD','HOUS','TRANS','HEALTH','RETPEN',]

models = {}
for target in targets:
    targetmodels = {}
    for race in races:
        starting = datetime.now()
        print("Fitting",target, "for", race)
        X = fmli[independents][fmli["model_"+race] == 1]
        y = fmli[target][fmli["model_"+race] == 1]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)        
        model = xgb.XGBRegressor()
        param_grid = {
            'objective': ['reg:squarederror'],  
            'max_depth': [3, 5], # [1, 3, 7],
            'learning_rate': [0.01],  # also called eta.  Tried 0.75, 0.5, 0.25, 0.1, 
            'subsample': [0.2, 0.7], #, 0.2, 0.3, 0.4, 0.5, 0.7
            'colsample_bytree':  [ 0.6, 0.8],  # 0.4, 0.6, 
            'lambda': [1, 2], # regularization 1, 2, 4
            'n_estimators': [1000], # 100, 250, 
            'eval_metric':['rmse'],
        }
        grid_search = GridSearchCV(model, param_grid, cv=5)
    
        grid_search.fit(X_train, y_train)
        print("Best set of hyperparameters: ", grid_search.best_params_)
        print("Best score: ", grid_search.best_score_)
        model = xgb.train(grid_search.best_params_, dtrain, num_boost_round=100)
        y_pred = model.predict(dtest)
        print("   ",r2_score(y_test, y_pred))
        targetmodels[race] = model
        duration = datetime.now() - starting
        print(f"Duration: {duration.seconds} seconds")
    models[target] = targetmodels

with open("blockgroup_regression_models.pkl", 'wb') as file:  
    pickle.dump(models, file)

print("Models saved")

Fitting TOTEXP for white
Best set of hyperparameters:  {'colsample_bytree': 0.6, 'eval_metric': 'rmse', 'lambda': 2, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1000, 'objective': 'reg:squarederror', 'subsample': 0.7}
Best score:  0.38891555070877076
    0.2705674171447754
Duration: 222 seconds
Fitting TOTEXP for black
Best set of hyperparameters:  {'colsample_bytree': 0.6, 'eval_metric': 'rmse', 'lambda': 2, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1000, 'objective': 'reg:squarederror', 'subsample': 0.2}
Best score:  0.4498751759529114
    0.34131187200546265
Duration: 204 seconds
Fitting TOTEXP for asian
Best set of hyperparameters:  {'colsample_bytree': 0.6, 'eval_metric': 'rmse', 'lambda': 2, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1000, 'objective': 'reg:squarederror', 'subsample': 0.2}
Best score:  0.35883508920669555
    0.2680951952934265
Duration: 203 seconds
Fitting TOTEXP for other
Best set of hyperparameters:  {'colsample_bytree': 0.6